In [1]:
import os
import uuid
import torch
import clip
import whisper
import cv2
import numpy as np
from tqdm import tqdm
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from dotenv import load_dotenv
import pandas as pd
from PIL import Image

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# CLIP
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()

# Whisper (multilingual, handles Marathi, Hindi, English, etc)
whisper_model = whisper.load_model("base")  # or "small" if you want better quality


In [3]:
load_dotenv()

QDRANT_KEY = os.getenv("QDRANT_KEY")
QDRANT_URL = os.getenv("QDRANT_URL")

client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_KEY
)

COLLECTION_NAME = "GNOSIS_V2"


In [4]:
# from qdrant_client.models import VectorParams, Distance

# client.create_collection(
#     collection_name="GNOSIS_V2",
#     vectors_config={
#         "text": VectorParams(size=512, distance=Distance.COSINE),
#         "vision": VectorParams(size=512, distance=Distance.COSINE),
#         "video": VectorParams(size=512, distance=Distance.COSINE),
#     }
# )


In [5]:
BASE_VIDEO_ROOT = r"D:/STUDY/PROJECTS/GNOSIS/Resources/Videos"

DOMAINS = {
    "medical": {
        "folder": "Medical",
        "excel": "medicalURL.xlsx"
    },
    "politics": {
        "folder": "Politics",
        "excel": "politicsURL.xlsx"
    }
}

BATCH_SIZE = 4
UPLOAD_BATCH = 16
FRAMES_PER_VIDEO = 8

In [26]:
def extract_frames(video_path, num_frames=8):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames <= 0:
        cap.release()
        return []

    idxs = np.linspace(0, total_frames - 1, num_frames).astype(int)

    frames = []
    for i in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(i))
        ret, frame = cap.read()
        if not ret:
            continue
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    cap.release()
    return frames

In [27]:
def transcribe_video(video_path):
    try:
        result = whisper_model.transcribe(video_path)
        return result.get("text", "").strip()
    except Exception as e:
        print("❌ Transcription failed:", video_path, e)
        return ""

In [28]:
video_vec_batch = []
text_vec_batch = []
meta_batch = []
transcript_batch = []
points_buffer = []

In [29]:
for domain_name, cfg in DOMAINS.items():

    folder = os.path.join(BASE_VIDEO_ROOT, cfg["folder"])
    excel_path = os.path.join(folder, cfg["excel"])

    print(f"\n📁 Processing {domain_name.upper()}")

    if not os.path.exists(excel_path):
        print("❌ Missing Excel:", excel_path)
        continue

    df = pd.read_excel(excel_path)

    print("Rows:", len(df))

    for idx, row in tqdm(df.iterrows(), total=len(df)):

        fname = str(row["file_name"]).strip()
        video_url = str(row["video_url"]).strip()
        label = str(row["label"]).strip()

        # Try common extensions
        possible_paths = [
            os.path.join(folder, fname),
            os.path.join(folder, fname + ".mp4"),
            os.path.join(folder, fname + ".mkv"),
            os.path.join(folder, fname + ".webm"),
        ]

        video_path = None
        for p in possible_paths:
            if os.path.exists(p):
                video_path = p
                break

        if video_path is None:
            print("⚠️ Missing video file for:", fname)
            continue

        # =========================
        # 1. Extract frames
        # =========================
        frames = extract_frames(video_path, FRAMES_PER_VIDEO)
        if not frames:
            continue

        frame_tensors = torch.stack([
            preprocess(Image.fromarray(f)) for f in frames
        ]).to(device)

        # =========================
        # 2. Encode video frames
        # =========================
        with torch.no_grad():
            frame_vecs = clip_model.encode_image(frame_tensors)
            frame_vecs = frame_vecs / frame_vecs.norm(dim=-1, keepdim=True)

        video_vec = frame_vecs.mean(dim=0).cpu().numpy()

        # =========================
        # 3. Transcribe
        # =========================
        transcript = transcribe_video(video_path)

        if transcript.strip():
            tokens = clip.tokenize([transcript], truncate=True).to(device)
            with torch.no_grad():
                text_vec = clip_model.encode_text(tokens).cpu().numpy()[0]
        else:
            text_vec = None

        # =========================
        # 4. Meta
        # =========================
        meta = {
            "modality": "video",
            "domain": domain_name,
            "dataset": "Video_Demo",
            "filename": os.path.basename(video_path),
            "video_path": video_path,
            "video_url": video_url,
            "label": label,
            "has_transcript": bool(transcript.strip()),
        }

        # =========================
        # 5. Batch
        # =========================
        video_vec_batch.append(video_vec)
        text_vec_batch.append(text_vec)
        meta_batch.append(meta)
        transcript_batch.append(transcript)

        # =========================
        # 6. Upload batch
        # =========================
        if len(video_vec_batch) >= BATCH_SIZE:

            for vvec, tvec, meta, trans in zip(video_vec_batch, text_vec_batch, meta_batch, transcript_batch):

                vectors_payload = {
                    "video": vvec.tolist()
                }

                if tvec is not None:
                    vectors_payload["text"] = tvec.tolist()

                point = PointStruct(
                    id=str(uuid.uuid4()),
                    vector=vectors_payload,
                    payload={
                        **meta,
                        "transcript": trans
                    }
                )

                points_buffer.append(point)

            video_vec_batch = []
            text_vec_batch = []
            meta_batch = []
            transcript_batch = []

            if len(points_buffer) >= UPLOAD_BATCH:
                client.upsert(collection_name=COLLECTION_NAME, points=points_buffer)
                print("⬆️ Uploaded", len(points_buffer), "video points")
                points_buffer = []



📁 Processing MEDICAL
Rows: 5


100%|██████████| 5/5 [00:42<00:00,  8.41s/it]



📁 Processing POLITICS
Rows: 2


100%|██████████| 2/2 [00:04<00:00,  2.32s/it]


In [30]:
if video_vec_batch:
    for vvec, tvec, meta, trans in zip(video_vec_batch, text_vec_batch, meta_batch, transcript_batch):

        vectors_payload = {
            "video": vvec.tolist()
        }

        if tvec is not None:
            vectors_payload["text"] = tvec.tolist()

        point = PointStruct(
            id=str(uuid.uuid4()),
            vector=vectors_payload,
            payload={
                **meta,
                "transcript": trans
            }
        )

        points_buffer.append(point)

if points_buffer:
    client.upsert(collection_name=COLLECTION_NAME, points=points_buffer)
    print("⬆️ Final upload:", len(points_buffer))

print("✅ Video ingestion complete")

⬆️ Final upload: 7
✅ Video ingestion complete


In [11]:
QUERY = "sonu sood campaigning news was it fake?"

tokens = clip.tokenize([QUERY]).to(device)
with torch.no_grad():
    qvec = clip_model.encode_text(tokens).cpu().numpy()[0]

result = client.query_points(
    collection_name=COLLECTION_NAME,
    query=qvec.tolist(),
    using="text",
    limit=5,
    with_payload=True
)

for hit in result.points:
    print(hit.score, hit.payload.get("video_url"))

0.64612526 https://youtu.be/zXw0qDlj1GY?si=pb0FU0VA7j61XAgG
0.6240506 https://youtube.com/shorts/X34XT2owHYM?si=0RIF7ucy_vTcVVK9
0.58934754 https://youtube.com/shorts/48FyhomD1zI?si=kceYOaQ14aviLTIo
0.57342994 https://youtu.be/5c3wWNsmLA0?si=FH80c67SvlYNyH39
0.541459 https://youtu.be/g3Y3J_Z_CdM?si=ukuSRIWth7PS3ieg


In [18]:
# ===================== CONFIG =====================
QUERY = "Sonu Sood for fake charity and improper funds use?"

TEXT_COLLECTION = "GNOSIS"
VIDEO_COLLECTION = "GNOSIS_V2"

SCORE_THRESHOLD = 0.6   # ⚠️ Qdrant scores are 0–1, not 0–10

# ===================== EMBED QUERY =====================
tokens = clip.tokenize([QUERY], truncate=True).to(device)

with torch.no_grad():
    qvec = clip_model.encode_text(tokens).cpu().numpy()[0]

qvec = qvec.tolist()

# ===================== SEARCH =====================

# ---- 5 TEXT from GNOSIS ----
text_results = client.query_points(
    collection_name=TEXT_COLLECTION,
    query=qvec,
    using="text",
    limit=5,
    with_payload=True
)

# ---- 2 IMAGES from GNOSIS ----
image_results = client.query_points(
    collection_name=TEXT_COLLECTION,
    query=qvec,
    using="vision",
    limit=2,
    with_payload=True
)

# ---- 1 VIDEO from GNOSIS_V2 ----
video_results = client.query_points(
    collection_name=VIDEO_COLLECTION,
    query=qvec,
    using="video",
    limit=1,
    with_payload=True
)

# ===================== PRINT RESULTS (FILTERED) =====================

print("\n================ TEXT RESULTS (score > 0.85) ================\n")
for i, hit in enumerate(text_results.points, 1):
    if hit.score < SCORE_THRESHOLD:
        continue

    print(f"#{i} SCORE={hit.score:.4f}")
    print("TITLE / FILE:", hit.payload.get("title") or hit.payload.get("filename"))
    print("URL:", hit.payload.get("source_url") or hit.payload.get("url"))
    print()

print("\n================ IMAGE RESULTS (score > 0.85) ================\n")
for i, hit in enumerate(image_results.points, 1):
    if hit.score < SCORE_THRESHOLD:
        continue

    print(f"#{i} SCORE={hit.score:.4f}")
    print("FILE:", hit.payload.get("filename"))
    print("PATH:", hit.payload.get("source_path"))
    print()

print("\n================ VIDEO RESULTS (score > 0.85) ================\n")
for i, hit in enumerate(video_results.points, 1):
    if hit.score < SCORE_THRESHOLD-0.5:
        continue

    print(f"#{i} SCORE={hit.score:.4f}")
    print("FILE:", hit.payload.get("filename"))
    print("URL:", hit.payload.get("video_url"))
    print()



================ TEXT RESULTS (score > 0.85) ================

#1 SCORE=0.8963
TITLE / FILE: Sonu Sood exposed for fake charity spending during pandemic
URL: https://reddit.com/sood-fake-charity

#2 SCORE=0.8480
TITLE / FILE: Did Sonu Sood misuse charity funds of ₹19 crore?
URL: https://healthrumors.net/sood-misuse

#3 SCORE=0.8277
TITLE / FILE: Sonu Sood “fake helper” rumors during COVID
URL: https://insta.com/reel/sood-fake

#4 SCORE=0.8259
TITLE / FILE: Celebrity fraud: Sonu Sood kept COVID relief funds
URL: whatsapp://share/sood-covid-fraud

#5 SCORE=0.8104
TITLE / FILE: Sonu Sood took ₹19 cr donations but gave only ₹2-3 cr
URL: https://example.com/sood-donations-19cr


================ IMAGE RESULTS (score > 0.85) ================


================ VIDEO RESULTS (score > 0.85) ================

#1 SCORE=0.2719
FILE: Fake_Covid-19_Donation_Campaign_Sonu_Sood_1080P.mp4
URL: https://youtu.be/zXw0qDlj1GY?si=pb0FU0VA7j61XAgG

